Customising MobilenetV2

In [3]:
!git clone https://github.com/coick4698/garbage_classification.git

from google.colab import drive
drive.mount('/content/drive')

%cd /content/garbage_classification

import sys
sys.path.append('/content/garbage_classification/src')

from dataset import get_dataloaders
from model import get_mobilenetv2
from train import train_model
from utils import plot_accuracy, plot_confusion_matrix

# 데이터 경로 고정
data_path = '/content/drive/MyDrive/iss_project/garbage_classification'

fatal: destination path 'garbage_classification' already exists and is not an empty directory.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/garbage_classification


In [4]:
# Exp.1 - Edit structure of FC layer -> increase accuracy
import torch
import torch.nn as nn
from torchvision.models import mobilenet_v2
from gradcam_utils import run_gradcam
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import cv2
import os

# 1. Defining Model
class CustomMobileNetV2_FC(nn.Module):
    def __init__(self, num_classes=12):
        super(CustomMobileNetV2_FC, self).__init__()
        base_model = mobilenet_v2(weights=None)
        self.features = base_model.features
        self.pool = nn.AdaptiveAvgPool2d(1)

        # FC layer
        self.classifier = nn.Sequential(
            nn.Linear(1280, 512), # (1280 -> num_classes) 를 (1280 -> 512 -> num_classes)로
            nn.ReLU(), # ReLU 추가
            nn.Linear(512, num_classes) # 중간 표현 학습 가능
        )

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# 2. load data
train_loader, val_loader, class_names = get_dataloaders(data_path, use_small=False)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 3. load model
model = CustomMobileNetV2_FC(num_classes=12).to(device)
train_acc, val_acc = train_model(model, train_loader, val_loader, num_epochs=10, device=device)

# 4. Accuracy Plot
plot_accuracy(train_acc, val_acc, title="Custom MobileNetV2-FC (Full Dataset)")

# 5. Confusion Matrix
plot_confusion_matrix(model, val_loader, class_names, device=device)

# 6. Precision / Recall / F1-score
all_preds, all_labels = [], []
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

print("Classification Report:")
print(classification_report(all_labels, all_preds, target_names=class_names))

# 7. Grad-CAM Visualization
target_layer = model.features[-1]
img_path = '/content/drive/MyDrive/iss_project/garbage_classification/plastic/plastic10.jpg'

gradcam_result = run_gradcam(
    model = model,
    target_layer = target_layer,
    img_path = img_path,
    use_cuda = torch.cuda.is_available()
)

plt.figure(figsize=(5, 5))
plt.title("Grad-CAM Visualization")
plt.imshow(gradcam_result)
plt.axis("off")
plt.show()


ModuleNotFoundError: No module named 'pytorch_grad_cam'